## Ready to be used ! 
You can run everything from scratch. We could not let all the vizualisations displayed, it was too heavy for Github 

:'-(

Hope you'll enjoy the project !

*French Team*

In [1]:
import pandas as pd
import altair as alt
from altair import expr, datum
from vega_datasets import data
import datetime as dt
#import streamlit as st


In [2]:
# connect gdrive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing our dataset

df = pd.read_csv('/content/drive/Shareddrives/Data vizualisation/top-2018-cleaned1.csv')
df.head()

In [ ]:
# we only focus on year 2018

df = df[df['published_date'].str.contains("2018")].reset_index(drop=True)
df

In [46]:
# replace NA by not available in category
df['category'] = df['category'].fillna('Not available', inplace = False)

In [ ]:
# we convert to datetime our published_date column and create 
# columns indicating month only for the slider

df['published_date'] = pd.to_datetime(df['published_date'])
df['month'] = df['published_date'].dt.to_period('M').astype(str)
df['month'] = pd.to_datetime(df['month'])
df['month_int'] = [df['month'][x].month for x in range(len(df['month']))]
df['year_int'] = [df['month'][x].year for x in range(len(df['month']))]

In [ ]:
# enable Altair to display the chart with > 5000 rows of data

alt.data_transformers.disable_max_rows()

In [ ]:
df.head()

In [ ]:
# GRAPH 1

click_domain = alt.selection_multi(encodings=['color']) 
daily_publi_per_domain = alt.Chart(df).mark_point().encode(
    x='published_date:T',
    y='fb_engagement:Q',
    color='Domain name:N',
    tooltip=['title:N', 'fb_engagement:Q', 'category:N'] 
).interactive().add_selection(click_domain).transform_filter(click_domain)

daily_publi_per_domain

# GRAPH 2

fake_news_daily = alt.Chart(df).mark_line().encode(
    x='published_date:T',
    y=alt.Y('count(titre):Q'),
    tooltip=['published_date:T', 'count(titre):Q', 'sum(fb_engagement):Q']
).interactive()

fake_news_daily | daily_publi_per_domain

In [ ]:
# top 10 fake news by category
df_top10_engagement_category = df.groupby(['category','title'])['fb_engagement'].sum().reset_index()
df_top10_engagement_category['Rank'] = df_top10_engagement_category.groupby('category')['fb_engagement'].rank('dense', ascending = False)
df_top10_engagement_category = df_top10_engagement_category[df_top10_engagement_category['Rank'] <= 10]
df_top10_engagement_category = df_top10_engagement_category.sort_values(['category','Rank'], ascending = False)

# adding a select to choose the category
all_category = df['category'].unique()
dropdown_cat = alt.binding_select(options= all_category)
select_category = alt.selection_single(fields=['category'], bind = dropdown_cat, name = 'Selector')

# barchart top 10 by cat
top10_articles_per_cat = alt.Chart(df_top10_engagement_category).mark_bar().encode(
    x=alt.X('fb_engagement:Q'),
    y=alt.Y('title:N',sort='-x'),
    color='category:N',
    tooltip='fb_engagement:Q'
).interactive().add_selection(
    select_category
).transform_filter(
    select_category
)

top10_articles_per_cat

In [ ]:
# adding a select to choose the origin
all_origin = df['Origin'].unique()
dropdown_origin = alt.binding_select(options= all_origin)
select_origin = alt.selection_single(fields=['Origin'], bind = dropdown_origin, name = 'Choose')

# Scatter plot with tooltip and interactive()
repartition_of_categ_per_origin = alt.Chart(df).mark_bar().encode(
    x=alt.X('Origin:N',sort ='-y'),
    y=alt.Y('count(category):Q'),
    color='category:N',
    tooltip=['count(category):Q', 'category:N']
).interactive().add_selection(
    select_origin
).transform_filter(
    select_origin
)

repartition_of_categ_per_origin

In [ ]:
# creating the slider

month_slider = alt.binding_range(min=1, max=12, step=1)
slider_selection_month = alt.selection_single(bind=month_slider, fields=['month_int'], name='slider')
click_cat_orig = alt.selection_multi(encodings=['color'])

# fake_news_daily
fake_news_daily = alt.Chart(df).mark_point().encode(
    x='published_date:T',
    y=alt.Y('count(titre):Q'),
    color=alt.Color('sum(fb_engagement)',scale=alt.Scale(scheme='goldred')),
    tooltip=['published_date:T', 'sum(fb_engagement):Q']
).add_selection(
    slider_selection_month
).transform_filter(
    slider_selection_month)
#.transform_filter(brush)

a = fake_news_daily.add_selection(click_cat_orig).transform_filter(click_cat_orig)

# engagement_per_origin

# building a specific df to get rank by origin by month top 10 
df_engagement_month_origin = df.groupby(['Origin','month_int', 'category'])['fb_engagement'].sum().reset_index()
df_engagement_month_origin['Rank'] = df_engagement_month_origin.groupby('month_int')['fb_engagement'].rank('dense', ascending = False)
df_engagement_month_origin = df_engagement_month_origin[df_engagement_month_origin['Rank'] <= 10]
df_engagement_month_origin = df_engagement_month_origin.sort_values('month_int')

# plotting it 
engagement_per_origin = alt.Chart(df_engagement_month_origin).mark_bar().encode(
    x='sum(fb_engagement):Q',
    y=alt.Y('Origin:N',sort='-x'),
    color='Origin:N',
    tooltip='sum(fb_engagement):Q'
).interactive().add_selection(click_cat_orig).transform_filter(click_cat_orig)

b = engagement_per_origin.add_selection(slider_selection_month).transform_filter(
    slider_selection_month)

# engagement_per_category

# building a specific df to get rank by category by month top 10 
df_engagement_month_category = df.groupby(['category','month_int', 'Origin'])['fb_engagement'].sum().reset_index()
df_engagement_month_category['Rank'] = df_engagement_month_category.groupby('month_int')['fb_engagement'].rank('dense', ascending = False)
df_engagement_month_category = df_engagement_month_category[df_engagement_month_category['Rank'] <= 10]
df_engagement_month_category = df_engagement_month_category.sort_values('month_int')

# plotting it 
engagement_per_category = alt.Chart(df_engagement_month_category).mark_bar().encode(
    x='sum(fb_engagement):Q',
    y=alt.Y('category:N',sort='-x'),
    color = 'category:N',
    tooltip='sum(fb_engagement):Q'
).interactive().add_selection(click_cat_orig).transform_filter(click_cat_orig)

engagement_per_category

c = engagement_per_category.add_selection(slider_selection_month).transform_filter(
    slider_selection_month)


a | b | c

In [ ]:
nb_word_engagement_cat = alt.Chart(df).mark_bar().encode(
    x=alt.X('Lenght of title:Q'),
    y=alt.Y('mean(fb_engagement):Q',sort='x'),
    color = alt.Color('mean(fb_engagement):Q',scale=alt.Scale(scheme='goldred')),
    tooltip=['mean(fb_engagement):Q', 'count(titre):Q']
).interactive().add_selection(
    select_category
).transform_filter(
    select_category
)

nb_word_engagement_origin = alt.Chart(df).mark_bar().encode(
    x=alt.X('Lenght of title:Q'),
    y=alt.Y('mean(fb_engagement):Q',sort='x'),
    color = alt.Color('mean(fb_engagement):Q',scale=alt.Scale(scheme='goldred')),
    tooltip=['mean(fb_engagement):Q', 'count(titre):Q']
).interactive().add_selection(
    select_origin
).transform_filter(
    select_origin
)

nb_word_engagement_cat & nb_word_engagement_origin



In [ ]:
click_day_of_week = alt.selection_multi(encodings=['color'])
daysofweek = ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"]

fake_news_DoW_cat = alt.Chart(df).mark_point().encode(
    x=alt.X('Day of the week:N',sort=daysofweek),
    y=alt.Y('count(titre):Q'),
    color=alt.Color('Day of the week:N', sort = daysofweek),
    tooltip=['count(titre):Q','sum(fb_engagement):Q']
).interactive().add_selection(click_day_of_week).interactive().add_selection(
    select_category
).transform_filter(
    select_category
)


fake_news_DoW_org = alt.Chart(df).mark_point().encode(
    x=alt.X('Day of the week:N',sort=daysofweek),
    y=alt.Y('count(titre):Q'),
    color=alt.Color('Day of the week:N', sort = daysofweek),
    tooltip=['count(titre):Q','sum(fb_engagement):Q']
).interactive().add_selection(click_day_of_week).interactive().add_selection(
    select_origin
).transform_filter(
    select_origin
)

fake_news_DoW_cat & fake_news_DoW_org

In [ ]:
# not used for the project

brush_weekday = alt.selection_interval(encodings=['x'])

nb_word_publication = alt.Chart(df).mark_bar().encode(
    x=alt.X('Lenght of title:Q'),
    y=alt.Y('count(title):Q',sort='x'),
    tooltip='count(title):Q'
).interactive().add_selection(brush_weekday).transform_filter(brush_weekday)

nb_word_engagement_2 = alt.Chart(df).mark_area().encode(
    x=alt.X('Lenght of title:Q'),
    y=alt.Y('sum(fb_engagement):Q'),
    color=alt.Color('Day of the week:N', sort = daysofweek),
    tooltip='sum(fb_engagement):Q'
).interactive().add_selection(brush_weekday).transform_filter(brush_weekday)

nb_word_publication & nb_word_engagement_2 